# SAE-79 - Visualisation t-SNE des Embeddings

**En tant que** data scientist  
**Je veux** visualiser les embeddings en 2D  
**Afin de** voir la structure des données

## Objectifs

- Appliquer t-SNE sur les embeddings de documents
- Visualiser la projection 2D
- Colorer par note (stars) pour voir la séparation
- Sauvegarder le graphique

## Inputs/Outputs

- **Input**: 
    - Embeddings: `outputs/doc_embeddings_w2v.npy`
    - Labels: `data/cleaned/reviews_clean.parquet`
- **Output**: 
    - Figure: `outputs/figures/tsne-embeddings.png`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from pathlib import Path

# Configuration visuelle
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Imports réussis")

## 1. Chargement des Données

In [ ]:
# Chemins
EMBEDDINGS_PATH = Path('../../outputs/doc_embeddings_w2v.npy')
REVIEWS_PATH = Path('../../data/cleaned/reviews_clean.parquet')
FIGURES_PATH = Path('../../outputs/figures')

# Créer dossier figures si inexistant
FIGURES_PATH.mkdir(parents=True, exist_ok=True)

print(f"📂 Embeddings: {EMBEDDINGS_PATH}")
print(f"📂 Reviews: {REVIEWS_PATH}")

In [ ]:
# Charger embeddings
if not EMBEDDINGS_PATH.exists():
    raise FileNotFoundError(f"❌ Fichier introuvable: {EMBEDDINGS_PATH}")

doc_embeddings = np.load(EMBEDDINGS_PATH)
print(f"✅ Embeddings chargées: {doc_embeddings.shape}")

# Charger labels (stars)
if not REVIEWS_PATH.exists():
    raise FileNotFoundError(f"❌ Fichier introuvable: {REVIEWS_PATH}")

df_reviews = pd.read_parquet(REVIEWS_PATH, columns=['stars'])
print(f"✅ Notes chargées: {len(df_reviews):,}")

# Vérification cohérence
if len(doc_embeddings) != len(df_reviews):
    print(f"⚠️ Attention: Mismatch dimensions ({len(doc_embeddings)} vs {len(df_reviews)}) - Utilisation de l'intersection")
    min_len = min(len(doc_embeddings), len(df_reviews))
    doc_embeddings = doc_embeddings[:min_len]
    df_reviews = df_reviews.iloc[:min_len]

## 2. Échantillonnage et t-SNE
t-SNE est très lent sur >10k points. Nous allons prendre un échantillon représentatif.

In [ ]:
# Paramètres
SAMPLE_SIZE = 5000  # Nombre de points à visualiser
RANDOM_SEED = 42

# Échantillonnage aléatoire
if len(doc_embeddings) > SAMPLE_SIZE:
    print(f"✂️ Échantillonnage de {SAMPLE_SIZE} points...")
    indices = np.random.choice(len(doc_embeddings), SAMPLE_SIZE, replace=False)
    X_sample = doc_embeddings[indices]
    y_sample = df_reviews['stars'].iloc[indices].values
else:
    X_sample = doc_embeddings
    y_sample = df_reviews['stars'].values

print(f"📊 Données prêtes pour t-SNE: {X_sample.shape}")

In [ ]:
print("⏳ Calcul t-SNE en cours (patience...)")

tsne = TSNE(
    n_components=2,
    perplexity=30,
    n_iter=1000,
    random_state=RANDOM_SEED,
    n_jobs=-1  # Utiliser tous les coeurs
)

X_embedded = tsne.fit_transform(X_sample)

print("✅ t-SNE terminé!")
print(f"   Output shape: {X_embedded.shape}")

## 3. Visualisation

In [ ]:
plt.figure(figsize=(14, 10))

# Créer le scatter plot
scatter = plt.scatter(
    X_embedded[:, 0],
    X_embedded[:, 1],
    c=y_sample,
    cmap='RdYlGn',  # Rouge (1) -> Vert (5)
    alpha=0.6,
    s=20,
    edgecolor='none'
)

# Ajouter la légende de couleur
cbar = plt.colorbar(scatter)
cbar.set_label('Rating (stars)', fontsize=12)

# Esthétique
plt.title('t-SNE Visualization of Document Embeddings\n(Color by Rating)', fontsize=16)
plt.xlabel('Dimension 1', fontsize=12)
plt.ylabel('Dimension 2', fontsize=12)
plt.grid(True, alpha=0.3)

# Sauvegarder
output_file = FIGURES_PATH / 'tsne-embeddings.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight')

print(f"💾 Graphique sauvegardé: {output_file}")
plt.show()

## 4. Analyse

### Observations
- **Structure Globale**: Regarder si les points forment des "nuages" distincts ou une masse continue.
- **Séparation des Sentiments**: 
    - Les points **verts** (5 étoiles) sont-ils séparés des points **rouges** (1 étoile) ?
    - Si oui, cela indique que les embeddings capturent bien l'information de sentiment.
    - Si non, les embeddings sont peut-être dominés par d'autres caractéristiques (thème, type de commerce, etc.).